# Ex13 习题集

## 1 课后作业-基础版

作业压缩包中给出了武汉市房价数据（示意）`wh_house_price.csv`和武汉市教育文化POI数据（示意）`wh_poi_edu_cul.geojson`,

请根据这两个文件，建立房价样本的教育指标：

- 1600米内小学个数：
- 1600米内中学个数：

将以上两个指标归一化(Min-Max归一化)后直接相加作为教育指标得分，并按照得分降序排列给出结果

In [2]:
import numpy
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 读取CSV文件为普通DataFrame
residential_df = pd.read_csv("wh_house_price.csv")
#print(residential_df.head())

# 将DataFrame转换为GeoDataFrame
geometry = [Point(xy) for xy in zip(residential_df['lon_gps'], residential_df['lat_gps'])]
residential_gdf = gpd.GeoDataFrame(residential_df, geometry=geometry)

# 设置坐标参考系统为WGS84
residential_gdf.crs = "EPSG:4326"
residential_gdf.to_crs("EPSG:3857", inplace=True)
# 读取GeoJSON文件
edu_gdf = gpd.read_file("wh_poi_edu_cul.geojson")

# 设置坐标参考系统为WGS84
edu_gdf.crs = "EPSG:4326"
edu_gdf.to_crs("EPSG:3857", inplace=True)

# print(residential_gdf.head())
# print(edu_gdf.head())

# 只保留小学的行
residential_buffer_gdf = residential_gdf.copy()
residential_buffer_gdf["buffer"] = residential_buffer_gdf.geometry.buffer(1600)

residential_buffer_gdf.set_geometry("buffer", inplace=True)
residential_buffer_gdf.crs = "EPSG:3857"
# Step 2: 空间连接
joined = gpd.sjoin(residential_buffer_gdf, edu_gdf, how='left', predicate='intersects')


# Step 3: 只保留小学
joined_primary = joined[joined['小类'].str.contains('小学', na=False)]

# Step 4: 按原小区 index 统计数量（这是关键）
school_count = joined_primary.groupby(joined_primary.index).size()

# Step 5: 写入原 GeoDataFrame
residential_gdf["count_primary_school"] = residential_gdf.index.map(school_count).fillna(0).astype(int)

print(residential_gdf[["name", "count_primary_school"]].head())

joined_middle = joined[joined['小类'].str.contains('中学', na=False)]
# print(joined_middle.head())

middle_count = joined_middle.groupby(joined_middle.index).size()
residential_gdf["count_middle_school"] = residential_gdf.index.map(middle_count).fillna(0).astype(int)

print(residential_gdf[["name", "count_middle_school"]].head())

def min_max(series):
    return (series - series.min()) / (series.max() - series.min())

residential_gdf["count_primary_school_normalized"] = min_max(residential_gdf["count_primary_school"])
residential_gdf["count_middle_school_normalized"] = min_max(residential_gdf["count_middle_school"])
residential_gdf["score"] = residential_gdf["count_primary_school_normalized"] + residential_gdf["count_middle_school_normalized"]

result = residential_gdf.sort_values(by="score", ascending=False)
print(result.head())




        name  count_primary_school
0       花惠社区                    12
1    保利海上五月花                     0
2  纯水岸东湖天樾小区                     1
3       佳海茗苑                     5
4       东亭花园                     5
        name  count_middle_school
0       花惠社区                    8
1    保利海上五月花                    1
2  纯水岸东湖天樾小区                    0
3       佳海茗苑                    2
4       东亭花园                    2
      id      name area  price  property_fee  green_rate  floor_area_ratio  \
92    93  中电广场尊荣国际  江汉区  21467           1.4      0.2232              9.70   
132  133       紫晶城  江汉区  22610           1.8      0.1850              5.49   
24    25       都会轩  江汉区  29736           4.5      0.2250              4.77   
39    40      台北名居  江岸区  17666           1.5      0.3000             10.40   
22    23      船舶广场  江岸区  18438           1.8      0.2200              1.80   

        lon_gps    lat_gps                          geometry  \
92   114.274593  30.576444  POINT (12720989.503 35778

## 2 课后作业-进阶版

根据课后作业-基础版的结果，完成以下要求：
1. 在前端搭建好Cesium框架（**只需要原始的默认框架即可**）
2. 在合适位置添加面板，面板包含两个文件选择框和一个按钮，两个文件选择框分别用于选择房价数据和教育文化POI数据。**面板不需要弄得太花里胡哨，不需要非常复杂的CSS，也不需要面板拖动显示隐藏等高级功能**
3. 点击按钮后，利用FastAPI接口，以房价数据的文件名和教育文化POI数据的文件名作为查询参数，在后端处理数据，再将带有教育指标的房价数据通过合适的方式返回至前端（提示：GeoJSON也是一种JSON）
4. 前端接收到后端返回的数据后，在Cesium的地图上显示并根据教育指标的值对点进行恰当的符号化

示例前端界面如下：

![界面示例](./界面示例.png)